In [26]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%autoreload 2

In [28]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utility_function import get_X_y , init_model, plot_loss
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense , Input , LSTM, Bidirectional, Dropout
from tensorflow.keras.models import Model , Sequential
from tensorflow.keras import regularizers
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [29]:
n_days = 1
length = 1
df  = pd.read_csv('data_2.csv').set_index('date').dropna()
n = 1515
s = 2610
df_test = df.iloc[s-length:s+90]
df_ = df.iloc[n:s]
df_true = df_test.copy()

In [30]:
scaler = MinMaxScaler()
scaler.fit(df_)
df_[df_.columns] =  scaler.transform(df_)
df_test[df_.columns] =  scaler.transform(df_test)

/Users/antoine/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/antoine/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/Users/antoine/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launch

In [31]:
X_train, y_train = get_X_y(df_, n_days, length, 'clf')

In [32]:
X_train = np.array(X_train)
X_train.shape = (X_train.shape[0], X_train.shape[2])

In [33]:
clf = LogisticRegression().fit(X_train, y_train)

/Users/antoine/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [34]:
prediction = clf.predict(df_test.drop(columns = 'up')[:-1])

In [21]:
accuracy = accuracy_score(df_test['up'][length:].values, prediction)
tn, fp, fn, tp = confusion_matrix(df_test['up'][length:].values, prediction).ravel()

In [22]:
accuracy

0.5666666666666667

In [23]:
tp/(tp+fn)

0.5625

In [24]:
tn / (tn+fp)

0.5714285714285714

In [35]:
prediction

array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [45]:
n_days = 1
length = 10
df  = pd.read_csv('data_2.csv').set_index('date').dropna()
#ipdb.set_trace()
n = 1880
s = 2520
df['close'] = df['close'].map(lambda x : np.exp(x))
score_trim = {}
for x in range(12) :
    s += 90
    n += 90
    score = []
    df_test = df.iloc[s:s+90]

    profit = 1
    mini = 1
    maxi = 1
    for i in range(1, len(df_test)) : 
        result = ((df_test['close'].iloc[i] - df_test['close'].iloc[i-1]) / df_test['close'].iloc[i-1]) + 1
        profit = profit * result
        if result < mini :
            mini = result
        if maxi < result :
             maxi = result
                
    score.append(accuracy)
    score.append(profit)
    score.append(mini)
    score.append(maxi)
    score_trim[x] = score

In [46]:
score_trim

{0: [0.5666666666666667,
  1.0811403744503039,
  0.8428994816087272,
  1.2316190464924501],
 1: [0.5666666666666667,
  1.2368427783198614,
  0.8976005373352774,
  1.1422964360284107],
 2: [0.5666666666666667,
  0.7636991557977252,
  0.8967653668208868,
  1.0921731396767407],
 3: [0.5666666666666667,
  0.9151202954533555,
  0.9261302404034122,
  1.0497484108008053],
 4: [0.5666666666666667,
  0.5471430176260813,
  0.8671511307639452,
  1.1172803093662986],
 5: [0.5666666666666667,
  1.560999435928594,
  0.9201515999964976,
  1.1754155175665129],
 6: [0.5666666666666667,
  1.7484607719202154,
  0.8575783994845456,
  1.1305789178140042],
 7: [0.5666666666666667,
  0.9067176888350073,
  0.8860396298987725,
  1.1545703884307477],
 8: [0.5666666666666667,
  0.9674974095780186,
  0.951386072544974,
  1.0996684803621701],
 9: [0.5666666666666667,
  0.8206880106893513,
  0.614238250602748,
  1.1823974075935249],
 10: [0.5666666666666667,
  1.2414969434854326,
  0.9135810102599939,
  1.127283610